# SO(3), the group of rotations in 3D space
## Introduction
SO(3), the 3D rotation group, is the group of all roations about the origin of three-dimensional Euclidian space $\mathbb{R}^3$ under the operation of composition ([wiki](https://en.wikipedia.org/wiki/3D_rotation_group)). 

- A rotation about the origin preserves the origin, euclidian distance and orientation.
- Composing two rotations results in another rotation.
- Every rotation has a unique inverse rotation.
- The identity map satisfies the definition of a rotation.

Therefore the set of all rotations is a group under composition.

## Representing rotations

In [2]:
# TODO matrices, vectors, quaternions
class SO3():
    def __init__(self, R):
        self.R = R

## SO(3) in Lie theory
The rotation group has a natural structure as a manifold for which the group operations are smoothly differentiable, so it is a Lie group.

### Lie algebra
Associated with every Lie group is its Lie algebra, a linear space of the same dimension as the Lie group, closed under a bilinear alternating product called the Lie bracket. 

The Lie algebra of SO(3) is denoted by $\mathfrak {so}(3)$ and consists of all skew-symmetric 3 × 3 matrices. 

The Lie algebra $\mathfrak {so}(3)$ is isomorphic to the Lie algebra  $\mathbb{R}^3$. Under this isomorphism, an Euleur vector $\omega \in \mathbb{R}^3$ corresponds to the linear map $\tilde{\omega} (u) = \omega \times u$.

In practice, a suitable basis to use $\mathfrak {so}(3)$ as a 3-dimensional vector space is
$$
\boldsymbol{L}_x = \begin{bmatrix}0&0&0\\0&0&-1\\0&1&0\end{bmatrix}, \quad
\boldsymbol{L}_y = \begin{bmatrix}0&0&1\\0&0&0\\-1&0&0\end{bmatrix}, \quad
\boldsymbol{L}_z = \begin{bmatrix}0&-1&0\\1&0&0\\0&0&0\end{bmatrix}.
$$
We can then identify any matrix of in this Lie algebra with an Euler vector $\omega = (x, y, z) \in \mathbb{R}^3$.

$$
\widehat{\boldsymbol{\omega}} =\boldsymbol{\omega}\cdot \boldsymbol{L} = x \boldsymbol{L}_x + y \boldsymbol{L}_y + z \boldsymbol{L}_z =\begin{bmatrix}0&-z&y\\z&0&-x\\-y&x&0\end{bmatrix} \in \mathfrak{so}(3)
$$

We can go from the skew symetric notations to euler angles and back using the skew and vee operators.

#### Logarithm map $\log : \operatorname{SO}(3) \to \mathfrak{so}(3)$


In [ ]:
def log_map(R):
    """
    Returns the skew matrix in lie algebra associated to the 
    SO3 rotation matrix R
    """
    